In [1]:
import numpy as np
import copy
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [29]:
dataset=StringIO("""Date,Open,High,Low,Close,Volume,Trade_count,Vwap
2015-12-01 09:00:00+00:00,118.88,118.94,118.88,118.94,1145,5,118.902052
2015-12-01 09:15:00+00:00,118.77,118.77,118.77,118.77,200,1,118.77
2015-12-01 09:30:00+00:00,118.69,118.69,118.6,118.6,900,4,118.61
2015-12-01 09:45:00+00:00,118.64,118.65,118.64,118.65,3580,5,118.648883
2015-12-01 10:00:00+00:00,118.65,118.65,118.55,118.55,1820,4,118.611538
2015-12-01 10:15:00+00:00,118.55,118.6,118.55,118.6,880,5,118.5625
2015-12-01 10:30:00+00:00,118.55,118.55,118.5,118.5,1878,5,118.513312
2015-12-01 10:45:00+00:00,118.59,118.72,118.59,118.72,2499,10,118.628431
2015-12-01 11:00:00+00:00,118.71,118.9,118.71,118.9,2842,11,118.86064
2015-12-01 11:15:00+00:00,118.87,118.87,118.87,118.87,300,2,118.87
2015-12-01 11:30:00+00:00,118.78,118.8,118.76,118.8,3914,22,118.785876
2015-12-01 11:45:00+00:00,118.8,118.99,118.77,118.9,7900,37,118.893542
2015-12-01 12:00:00+00:00,118.88,118.98,118.84,118.84,6540,34,118.922648
2015-12-01 12:15:00+00:00,118.82,118.84,118.77,118.77,5603,28,118.804962
2015-12-01 12:30:00+00:00,118.77,118.89,118.76,118.88,7612,31,118.824002
""")
df = pd.read_table(dataset, sep=",")

#ip = np.array([ [1,2,3],[6,8,9],[3,4,5],[4,7,8],[4,2,5],[5,7,4] ])
#op = np.array([[2,8,4,7,2,4]])
#op = op.reshape(6,1)
ip = np.array([ [1],  [2],  [0],  [2],  [0],  [1],  [2],  [1] ])
op = np.array([ [300],[100],[200],[100],[200],[300],[100],[300] ])
num_steps = 3
num_features = 3
#ip_shaped = np.reshape(ip, newshape=(-1, num_steps, num_features))

#X = np.array([ [1,2,3, 4, 5, 6] ])
#Y = np.array([[2,3,4,5,6,7]])
print(ip)
print("--")
#ip= np.tile(ip,(50,1))
#op = np.tile(op,(50,1))
print(len(op))
#ip = np.array([ [1],[1],[1],[1],[1],[2],[1],[2]])
#op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T

ip = np.array([ [1,1,2],[1,2,1],[1,0,2],[1,2,1],[1,0,2],[1,1,2],[1,2,1],[1,1,2]])
op = np.array([ [1,    0,    1,    0,    1,    1,    0,    1 ]]).T
#ip = np.array([ [1,1,1,1,1],[0,2,2,2,2],[1,3,3,3,3],[1,4,4,4,4],[0,5,5,5,5],[0,6,6,6,6],[1,7,7,7,7],[0,8,8,8,8]])
#op = np.array([ [1,          0,          1,          1,          0,          0,          1,          0 ]]).T
_, numFeats = ip.shape

[[1]
 [2]
 [0]
 [2]
 [0]
 [1]
 [2]
 [1]]
--
8


In [175]:
def lstm_data_transform(x_data, y_data, timeSteps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + timeSteps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        #print(x_data[i:end_ix])
        # Get only the last element of the sequency for y
        #print(y_data[end_ix])
        seq_y = y_data[i:end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
        display(X)
        print("--")
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

def lstm_data_time(x_data, y_data, timeSteps=2):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    #print(x_data.shape[0])
    x_array = []
    y_array = []

    for i in range(x_data.shape[0]):
        my2, mx2 = list(), list()
        # compute a new (sliding window) index
        end_ix = i + timeSteps

        # if index is larger than the size of the dataset, we stop
        #print(end_ix)
        if end_ix >= x_data.shape[0]:
            break

        for j in range(i, i+timeSteps):
            my1, mx1 = list(), list()
            # Get a sequence of data for x
            seq_X = x_data[j]
            # Get only the last element of the sequency for y
            seq_y = y_data[j]
            # Append the list with sequencies
            for k in range(len(seq_X)):
                my, mx = list(), list()
                mx.append(seq_X[k])
                mx1.append(mx)
                
            mx2.append(mx1)
            my2.append(seq_y)
            
        X.append(mx2)
        y.append(my2)

        #X.append(np.array(mx1))
        #y.append(np.array(my))
        print("--")
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)

    return x_array, y_array
#3 3 1 want
#3 3 3 now

In [195]:
timeSteps = 3
ip = np.array([ [1,1,1,2],[1,1,2,1],[1,1,0,2],[1,1,2,1],[1,1,0,2],[1,1,1,2],[1,1,2,1],[1,1,1,2]])
print(ip.shape)
ipt,opt=lstm_data_time(ip,op, timeSteps)
display(ipt)
print(ipt.shape)
print(ipt[0].shape)
print(ipt[0])
print(opt.shape)

print("====")
print(opt[0])
# batchSize = len(ipt)
# [[[1.]
#   [0.]
#   [0.]]

#  [[1.]
#   [0.]
#   [0.]]

#  [[1.]
#   [0.]
#   [0.]]

(8, 4)
--
--
--
--
--


array([[[[1],
         [1],
         [1],
         [2]],

        [[1],
         [1],
         [2],
         [1]],

        [[1],
         [1],
         [0],
         [2]]],


       [[[1],
         [1],
         [2],
         [1]],

        [[1],
         [1],
         [0],
         [2]],

        [[1],
         [1],
         [2],
         [1]]],


       [[[1],
         [1],
         [0],
         [2]],

        [[1],
         [1],
         [2],
         [1]],

        [[1],
         [1],
         [0],
         [2]]],


       [[[1],
         [1],
         [2],
         [1]],

        [[1],
         [1],
         [0],
         [2]],

        [[1],
         [1],
         [1],
         [2]]],


       [[[1],
         [1],
         [0],
         [2]],

        [[1],
         [1],
         [1],
         [2]],

        [[1],
         [1],
         [2],
         [1]]]])

(5, 3, 4, 1)
(3, 4, 1)
[[[1]
  [1]
  [1]
  [2]]

 [[1]
  [1]
  [2]
  [1]]

 [[1]
  [1]
  [0]
  [2]]]
(5, 3, 1)
====
[[1]
 [0]
 [1]]


In [5]:
def sigmoid(x, derivative=False):
    """
    Computes the element-wise sigmoid activation function for an array x.

    Args:
     `x`: the array where the function is applied
     `derivative`: if set to True will return the derivative instead of the forward pass
    """
    x_safe = x + 1e-12
    f = 1 / (1 + np.exp(-x_safe))
    
    if derivative: # Return the derivative of the function evaluated at x
        return f * (1 - f)
    else: # Return the forward pass of the function at x
        return f



def init_orthogonal(param):
    """
    Initializes weight parameters orthogonally.
    This is a common initiailization for recurrent neural networks.
    
    Refer to this paper for an explanation of this initialization:
    https://arxiv.org/abs/1312.6120
    """
    if param.ndim < 2:
        raise ValueError("Only parameters with 2 or more dimensions are supported.")

    rows, cols = param.shape
    
    new_param = np.random.randn(rows, cols)
    
    if rows < cols:
        new_param = new_param.T
    
    # Compute QR factorization
    q, r = np.linalg.qr(new_param)
    
    # Make Q uniform according to https://arxiv.org/pdf/math-ph/0609050.pdf
    d = np.diag(r, 0)
    ph = np.sign(d)
    q *= ph

    if rows < cols:
        q = q.T
    
    new_param = q
    
    return new_param

  
def tanh(x, derivative=False):
    """
    Computes the element-wise tanh activation function for an array x.

    Args:
     `x`: the array where the function is applied
     `derivative`: if set to True will return the derivative instead of the forward pass
    """
    x_safe = x + 1e-12
    f = (np.exp(x_safe)-np.exp(-x_safe))/(np.exp(x_safe)+np.exp(-x_safe))
    
    if derivative: # Return the derivative of the function evaluated at x
        return 1-f**2
    else: # Return the forward pass of the function at x
        return f
    
    
def softmax(x, derivative=False):
    """
    Computes the softmax for an array x.
    
    Args:
     `x`: the array where the function is applied
     `derivative`: if set to True will return the derivative instead of the forward pass
    """
    x_safe = x + 1e-12
    f = np.exp(x_safe) / np.sum(np.exp(x_safe))
    
    if derivative: # Return the derivative of the function evaluated at x
        pass # We will not need this one
    else: # Return the forward pass of the function at x
        return f
    
def clip_gradient_norm(grads, max_norm=0.25):
    """
    Clips gradients to have a maximum norm of `max_norm`.
    This is to prevent the exploding gradients problem.
    """ 
    # Set the maximum of the norm to be of type float
    max_norm = float(max_norm)
    total_norm = 0
    
    # Calculate the L2 norm squared for each gradient and add them to the total norm
    for grad in grads:
        grad_norm = np.sum(np.power(grad, 2))
        total_norm += grad_norm
    
    total_norm = np.sqrt(total_norm)
    
    # Calculate clipping coeficient
    clip_coef = max_norm / (total_norm + 1e-6)
    
    # If the total norm is larger than the maximum allowable norm, then clip the gradient
    if clip_coef < 1:
        for grad in grads:
            grad *= clip_coef
    
    return grads

def update_parameters(params, grads, lr=1e-3):
    # Take a step
    for param, grad in zip(params, grads):
        param -= lr * grad
    
    return params



In [196]:
od=1
def init_lstm(hd, timeSteps):
    """
    Initializes our LSTM network.
    
    Args:
     `hidden_size`: the dimensions of the hidden state
     `vocab_size`: the dimensions of our vocabulary
     `z_size`: the dimensions of the concatenated input 
    """
    z=hd+timeSteps
    
    wxi = 2*np.random.random((1,numFeats)) - 1
    wxf = 2*np.random.random((1,numFeats)) - 1
    wxc = 2*np.random.random((1,numFeats)) - 1
    wxo = 2*np.random.random((1,numFeats)) - 1

    whi= 2*np.random.random((hd,numFeats)) - 1
    whf= 2*np.random.random((hd,numFeats)) - 1
    whc= 2*np.random.random((hd,numFeats)) - 1
    who = 2*np.random.random((hd,numFeats)) - 1

    wy = 2*np.random.random((hd,1)) - 1

    
    bi = np.zeros((numFeats, 1))
    bf = np.zeros((hd, 1))
    bc = np.zeros((hd, 1))
    bo = np.zeros((hd, 1))
    
    by = np.zeros((hd, 1))
    

    return wxi, wxf, wxc, wxo, whi, whf, whc, who, wy, bi, bf, bc, bo, by


params = init_lstm(6, timeSteps)
print('wxi:', params[0].shape)
print('wxf:', params[1].shape)
print('wxc:', params[2].shape)
print('wxo:', params[3].shape)

print('whi:', params[4].shape)
print('whf:', params[5].shape)
print('whc:', params[6].shape)
print('who:', params[7].shape)

print('wy:', params[8].shape)

print('bi:', params[9].shape)
print('bf:', params[10].shape)
print('bc:', params[11].shape)
print('bo:', params[12].shape)
print('by:', params[13].shape)

for param in params:
    assert param.ndim == 2, \
        'all parameters should be 2-dimensional '\
        '(hint: a dimension can simply have size 1)'

wxi: (1, 3)
wxf: (1, 3)
wxc: (1, 3)
wxo: (1, 3)
whi: (6, 3)
whf: (6, 3)
whc: (6, 3)
who: (6, 3)
wy: (6, 1)
bi: (3, 1)
bf: (6, 1)
bc: (6, 1)
bo: (6, 1)
by: (6, 1)


In [7]:
import numpy as np

# Set seed such that we always get the same dataset
# (this is a good idea in general)
np.random.seed(42)

def generate_dataset(num_sequences=2**8):
    """
    Generates a number of sequences as our dataset.
    
    Args:
     `num_sequences`: the number of sequences to be generated.
     
    Returns a list of sequences.
    """
    samples = []
    
    for _ in range(num_sequences): 
        num_tokens = np.random.randint(1, 12)
        sample = ['1'] * num_tokens + ['0'] * num_tokens
        samples.append(sample)
        
    return samples


sequences = generate_dataset()

print('A single sample from the generated dataset:')
print(sequences[0])


A single sample from the generated dataset:
['1', '1', '1', '1', '1', '1', '1', '0', '0', '0', '0', '0', '0', '0']


In [238]:
# Size of concatenated hidden + input vector
hd = 50
numFeats = ind = 4
z = hd + ind

def init_lstm(hd, ind, z):
    """
    Initializes our LSTM network.
    
    Args:
     `hidden_size`: the dimensions of the hidden state
     `vocab_size`: the dimensions of our vocabulary
     `z_size`: the dimensions of the concatenated input 
    """
    # Weight matrix (forget gate)
    W_f = np.zeros((hd, z))
    # Weight matrix (input gate)
    W_i = np.zeros((hd, z))
    # Weight matrix (candidate)
    W_g = np.zeros((hd, z))
   
    # Bias for forget gate
    b_f = np.zeros((hd, 1))

    
    # Bias for input gate
    b_i = np.zeros((hd, 1))


    # Bias for candidate
    b_g = np.zeros((hd, 1))

    # Weight matrix of the output gate
    W_o = np.zeros((hd, z))
    
    # Bias for output gate
    b_o = np.zeros((hd, 1))

    # Weight matrix relating the hidden-state to the output
    W_v = np.zeros((1, hd))
    
    # Bias for logits
    b_v = np.zeros((1, 1))
    
    # Initialize weights according to https://arxiv.org/abs/1312.6120
    W_f = init_orthogonal(W_f)
    W_i = init_orthogonal(W_i)
    W_g = init_orthogonal(W_g)
    W_o = init_orthogonal(W_o)
    W_v = init_orthogonal(W_v)

    return W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v

In [240]:
def forward(inputs, h_prev, C_prev, p):
    """
    Arguments:
    x -- your input data at timestep "t", numpy array of shape (n_x, m).
    h_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    C_prev -- Memory state at timestep "t-1", numpy array of shape (n_a, m)
    p -- python list containing:
                        W_f -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        b_f -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        W_i -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        b_i -- Bias of the update gate, numpy array of shape (n_a, 1)
                        W_g -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        b_g --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                        W_o -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        b_o --  Bias of the output gate, numpy array of shape (n_a, 1)
                        W_v -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_v, n_a)
                        b_v -- Bias relating the hidden-state to the output, numpy array of shape (n_v, 1)
    Returns:
    z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s -- lists of size m containing the computations in each forward pass
    outputs -- prediction at timestep "t", numpy array of shape (n_v, m)
    """
    assert h_prev.shape == (hd, 1)
    assert C_prev.shape == (hd, 1)

    # First we unpack our parameters
    W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v = p
    
    # Save a list of computations for each of the components in the LSTM
    x_s, z_s, f_s, i_s,  = [], [] ,[], []
    g_s, C_s, o_s, h_s = [], [] ,[], []
    v_s, output_s =  [], [] 
    
    # Append the initial cell and hidden state to their respective lists
    h_s.append(h_prev)
    C_s.append(C_prev)
    
    for x in inputs:
        # Concatenate input and hidden state
        z = np.row_stack((h_prev, x))
        z_s.append(z)
        
        # Calculate forget gate
        f = sigmoid(np.dot(W_f, z) + b_f)
        f_s.append(f)
        
        # Calculate input gate
        i = sigmoid(np.dot(W_i, z) + b_i)
        i_s.append(i)
        
        # Calculate candidate
        g = tanh(np.dot(W_g, z) + b_g)
        g_s.append(g)
        
        # Calculate memory state
        C_prev = C_prev * f + g * i
        C_s.append(C_prev)
        
        # Calculate output gate
        o = sigmoid(np.dot(W_o, z) + b_o)
        o_s.append(o)
        
        # Calculate hidden state
        h_prev = o * tanh(C_prev)
        h_s.append(h_prev)

        # Calculate logits
        v = np.dot(W_v, h_prev) + b_v
        v_s.append(v)

        # Calculate softmax
        #output = softmax(v)
        output_s.append(v)

    return z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, output_s


# Get first sentence in test set
inputs, targets = test_set[1]

# Initialize hidden state as zeros
h = np.zeros((hd, 1))
c = np.zeros((hd, 1))

# Forward pass
z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, outputs = forward(ipt[0], h, c, params)

#output_sentence = [idx_to_word[np.argmax(output)] for output in outputs]
print('Input:')
print(ipt[0])

print('Target:')
print(opt[0])

print('Predicted:')
print(np.array(v_s))
# print([idx_to_word[np.argmax(output)] for output in outputs])




Input:
[[[1]
  [1]
  [1]
  [2]]

 [[1]
  [1]
  [2]
  [1]]

 [[1]
  [1]
  [0]
  [2]]]
Target:
[[1]
 [0]
 [1]]
Predicted:
[[[1.0006564 ]]

 [[0.99887137]]

 [[0.99909264]]]


In [235]:
def backward(z, f, i, g, C, o, h, v, outputs, targets, p = params):
    """
    Arguments:
    z -- your concatenated input data  as a list of size m.
    f -- your forget gate computations as a list of size m.
    i -- your input gate computations as a list of size m.
    g -- your candidate computations as a list of size m.
    C -- your Cell states as a list of size m+1.
    o -- your output gate computations as a list of size m.
    h -- your Hidden state computations as a list of size m+1.
    v -- your logit computations as a list of size m.
    outputs -- your outputs as a list of size m.
    targets -- your targets as a list of size m.
    p -- python list containing:
                        W_f -- Weight matrix of the forget gate, numpy array of shape (n_a, n_a + n_x)
                        b_f -- Bias of the forget gate, numpy array of shape (n_a, 1)
                        W_i -- Weight matrix of the update gate, numpy array of shape (n_a, n_a + n_x)
                        b_i -- Bias of the update gate, numpy array of shape (n_a, 1)
                        W_g -- Weight matrix of the first "tanh", numpy array of shape (n_a, n_a + n_x)
                        b_g --  Bias of the first "tanh", numpy array of shape (n_a, 1)
                        W_o -- Weight matrix of the output gate, numpy array of shape (n_a, n_a + n_x)
                        b_o --  Bias of the output gate, numpy array of shape (n_a, 1)
                        W_v -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_v, n_a)
                        b_v -- Bias relating the hidden-state to the output, numpy array of shape (n_v, 1)
    Returns:
    loss -- crossentropy loss for all elements in output
    grads -- lists of gradients of every element in p
    """

    # Unpack parameters
    W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v = p

    # Initialize gradients as zero
    W_f_d = np.zeros_like(W_f)
    b_f_d = np.zeros_like(b_f)

    W_i_d = np.zeros_like(W_i)
    b_i_d = np.zeros_like(b_i)

    W_g_d = np.zeros_like(W_g)
    b_g_d = np.zeros_like(b_g)

    W_o_d = np.zeros_like(W_o)
    b_o_d = np.zeros_like(b_o)

    W_v_d = np.zeros_like(W_v)
    b_v_d = np.zeros_like(b_v)
    
    # Set the next cell and hidden state equal to zero
    dh_next = np.zeros_like(h[0])
    dC_next = np.zeros_like(C[0])
        
    # Track loss
    loss = 0
    
    for t in reversed(range(len(outputs))):
        # Compute the cross entropy
        loss += -np.mean(np.log(outputs[t]) * targets[t])

        # Get the previous hidden cell state
        C_prev= C[t-1]

        # Compute the derivative of the relation of the hidden-state to the output gate
        dv = np.copy(outputs[t])
        dv[np.argmax(targets[t])] -= 1

        # Update the gradient of the relation of the hidden-state to the output gate
        W_v_d += np.dot(dv, h[t].T)
        b_v_d += dv

        # Compute the derivative of the hidden state and output gate
        dh = np.dot(W_v.T, dv)
        dh += dh_next
        do = dh * tanh(C[t])
        do = sigmoid(o[t], derivative=True)*do

        # Update the gradients with respect to the output gate
        W_o_d += np.dot(do, z[t].T)
        b_o_d += do

        # Compute the derivative of the cell state and candidate g
        dC = np.copy(dC_next)
        dC += dh * o[t] * tanh(tanh(C[t]), derivative=True)
        dg = dC * i[t]
        dg = tanh(g[t], derivative=True) * dg

        # Update the gradients with respect to the candidate
        W_g_d += np.dot(dg, z[t].T)
        b_g_d += dg

        # Compute the derivative of the input gate and update its gradients
        di = dC * g[t]
        di = sigmoid(i[t], True) * di
        W_i_d += np.dot(di, z[t].T)
        b_i_d += di

        # Compute the derivative of the forget gate and update its gradients
        df = dC * C_prev
        df = sigmoid(f[t]) * df
        W_f_d += np.dot(df, z[t].T)
        b_f_d += df

        # Compute the derivative of the input and update the gradients of the previous hidden and cell state
        dz = (np.dot(W_f.T, df)
             + np.dot(W_i.T, di)
             + np.dot(W_g.T, dg)
             + np.dot(W_o.T, do))
        dh_prev = dz[:hd, :]
        dC_prev = f[t] * dC
        
    grads= W_f_d, W_i_d, W_g_d, W_o_d, W_v_d, b_f_d, b_i_d, b_g_d, b_o_d, b_v_d
    
    # Clip gradients
    grads = clip_gradient_norm(grads)
    
    return loss, grads


# Perform a backward pass
loss, grads = backward(z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, outputs, opt[0], params)
print(outputs)
print(opt[0])
print('We get a loss of:')
print(loss)

[array([[0.99959403]]), array([[0.99930368]]), array([[1.0007546]])]
[[1]
 [0]
 [1]]
We get a loss of:
0.9986085745539479


In [241]:
class Dataset(data.Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        # Return the size of the dataset
        return len(self.targets)

    def __getitem__(self, index):
        # Retrieve inputs and targets at the given index
        X = self.inputs[index]
        y = self.targets[index]

        return X, y

validSet=Dataset(ipt, opt)

trainSet=Dataset(ipt, opt)

# Hyper-parameters
num_epochs = 2000

# Initialize a new network
hd=50
z_size = hd + numFeats # Size of concatenated hidden + input vector
params = init_lstm(hd, numFeats, z=z_size)

# Initialize hidden state as zeros
hidden_state = np.zeros((hd, 1))

# Track loss
training_loss, validation_loss = [], []

# For each epoch
for i in range(num_epochs):
    
    # Track loss
    epoch_training_loss = 0
    epoch_validation_loss = 0

    # For each sentence in validation set
    for inputs, targets in validSet:
        
        # Initialize hidden state and cell state as zeros
        h = np.zeros((hd, 1))
        c = np.zeros((hd, 1))

        # Forward pass
        zs, fs, ins, gs, Cs, os, hs, vs, outputs = forward(inputs, h, c, params)
        
        # Backward pass
        loss, _ = backward(zs, fs, ins, gs, Cs, os, hs, vs, outputs, targets, params)
        
        # Update loss
        epoch_validation_loss += loss
    
    # For each sentence in training set
    for inputs, targets in trainSet:
        
        # Initialize hidden state and cell state as zeros
        h = np.zeros((hd, 1))
        c = np.zeros((hd, 1))

        # Forward pass
        zs, fs, ins, gs, Cs, os, hs, vs, outputs = forward(inputs, h, c, params)
        
        # Backward pass
        loss, grads = backward(zs, fs, ins, gs, Cs, os, hs, vs, outputs, targets, params)
        
        # Update parameters
        params = update_parameters(params, grads, lr=1e-1)
        
        # Update loss
        epoch_training_loss += loss
                
    # Save loss for plot
    training_loss.append(epoch_training_loss/len(training_set))
    validation_loss.append(epoch_validation_loss/len(validation_set))

    # Print loss every 10 epochs
    if i % 10 == 0:
        print(f'Epoch {i}, training loss: {training_loss[-1]}, validation loss: {validation_loss[-1]}')

Epoch 0, training loss: 0.04214851505304882, validation loss: 0.3986063178449619
Epoch 10, training loss: 0.029931518905172066, validation loss: 0.24407418824899693
Epoch 20, training loss: 0.029821800214628186, validation loss: 0.24397066952403917
Epoch 30, training loss: 0.02980736031984061, validation loss: 0.2442878639106558
Epoch 40, training loss: 0.02972276003941205, validation loss: 0.24370422959477672
Epoch 50, training loss: 0.029615400921454472, validation loss: 0.24278632933476665
Epoch 60, training loss: 0.02951804351057778, validation loss: 0.24190348269729608
Epoch 70, training loss: 0.029441010994751642, validation loss: 0.24118274587381033
Epoch 80, training loss: 0.029384124719823743, validation loss: 0.2406370808210111
Epoch 90, training loss: 0.0293437960209227, validation loss: 0.2402400247216471
Epoch 100, training loss: 0.02931600841867413, validation loss: 0.23995753354949934
Epoch 110, training loss: 0.029297322914925657, validation loss: 0.23975923249848796
Ep

Epoch 980, training loss: 0.029264408276876156, validation loss: 0.23909036530517436
Epoch 990, training loss: 0.029263557158500503, validation loss: 0.23908601054002415
Epoch 1000, training loss: 0.02926270591833799, validation loss: 0.23908167125764668
Epoch 1010, training loss: 0.02926185542644095, validation loss: 0.23907735032446056
Epoch 1020, training loss: 0.029261006573299084, validation loss: 0.23907305071274018
Epoch 1030, training loss: 0.02926016027002464, validation loss: 0.2390687755048693
Epoch 1040, training loss: 0.029259317448461067, validation loss: 0.23906452789749286
Epoch 1050, training loss: 0.029258479061212282, validation loss: 0.2390603112055403
Epoch 1060, training loss: 0.029257646081589737, validation loss: 0.23905612886609506
Epoch 1070, training loss: 0.029256819503474776, validation loss: 0.23905198444208142
Epoch 1080, training loss: 0.029256000341093803, validation loss: 0.2390478816257375
Epoch 1090, training loss: 0.02925518962870421, validation los

Epoch 1950, training loss: 0.029321057123244197, validation loss: 0.23939854627949217
Epoch 1960, training loss: 0.029323787729831085, validation loss: 0.2394134885689673
Epoch 1970, training loss: 0.029326548273515024, validation loss: 0.23942858909289208
Epoch 1980, training loss: 0.029329337748630408, validation loss: 0.23944384108069156
Epoch 1990, training loss: 0.029332155153943645, validation loss: 0.2394592377731591


In [242]:
zs, fs, ins, gs, Cs, os, hs, vs, outputs, outputs = forward(ipt[1], h, c, params)
print(v_s)
print(opt[1])

ValueError: not enough values to unpack (expected 10, got 9)